In [19]:
import pandas as pd
import numpy as np
import time
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/danterangel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:

str_replaces_list = [
    ['*',' '],
    ['.',' '],
    ['-',' '],
    ['-',' '],
    ['01 ',''],
    ['02 ',''],
    ['03 ',''],
    ['04 ',''],
    ['05 ',''],
    ['06 ',''],
    ['07 ',''],
    ['08 ',''],
    ['09 ',''],
    ['/', ' '],
    ['AMEX','American Express'],
    ['IRS','Internal Revenue Service'],
    ['SYNCB','SYNCHRONY'],
    ['JPMCB - CARD SERVICE','JPMCB - CARD'],
    ['JPMCB','JPMChase Bank'],
    ['JPM CB','JPMChase Bank'],
    ['5TH 3RD','Fifth third'],
    ['5/3','Fifth third'],
    ['1NEVADACU','One Nevada'],
    ['CBNA','Citibank, N.A.'],
    ['TOYOTA MOTOR CREDIT','Toyota Financial Services'],
    ['ADVANT','Advantage'],
    ['CAPONE','Capital one'],
    ['CRED','Credit'],
    ['Ins ', 'Insurance '],
    ['P & C', 'Property & Casualty'],
    ['P&C', 'Property & Casualty'],
    ['P& C', 'Property & Casualty'],
    ['PROP & CAS', 'PROPERTY & CASUALTY'],
    ['INS ', 'INSURANCE '],
    ['CORP ', 'CORPORATE '],
    ['Ins. Co.', 'Insurance Companty'],
    ['Cas.', 'Casuality'],
    ['CAS UALTY', 'CASUALTY'],
    ['CAS CO', 'CASUALITY COMPANY'],
    ['COUNTRY INS', 'COMPANY INSURANCE'],
    ['FBSD','1st Financial Bank'],
    ['121FCU','Financial Credit Union'],
    ['121 FCU','Financial Credit Union'],
    ['1st Comm CU','first Community credit union'],
    ['1ST COMM','first Community'],
    ['1ST FEDERAL','first federal'],
    ['1ST FINANCIAL BANK ','first financial bank']
]
#str_replaces_list = []
b_words = [
    'insurance',
    'insurances',
    'bank',
    'counties',
    'credit',
    'finance',
    'financial',
    'services',
    'service',
    'credit',
    'service',
    'corporate',
    'company',
    'casuality',
    'exchange',
    'association',
    'cooperative',
    'property'
]

b_words = stopwords.words('english') + b_words

In [21]:
def str_replace(string, str_replaces_list):
    for i in str_replaces_list:
        string = string.replace(i[0], i[1])
    return string.lower()

In [22]:
def ngrams(string, n=12):
    string = string.encode("ascii", errors="ignore").decode() #remove non ascii chars
    string = string.lower() #make lower case
    string = str_replace(string,str_replaces_list)
    chars_to_remove = [")","(",".","|","[","]","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string) #remove the list of chars defined above
    string = string.title() # normalise case - capital at start of each word
    string = re.sub(' +',' ',string).strip() # get rid of multiple spaces and replace with a single space
    string = ' '+ string +' ' # pad names for ngrams...
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

In [23]:
def train_vectorize(org_name_clean, unique_org,analyzer= 'char_wb'):
    t1 = time.time()
    print('=====================================================Vecorizing the data - this could take a few minutes for large datasets... =====================================================')
    vectorizer = TfidfVectorizer(min_df=1, analyzer=analyzer, stop_words=b_words)#, lowercase=False)
    tfidf = vectorizer.fit_transform(org_name_clean)
    t = time.time()-t1
    print("=====================  COMPLETED IN:", t)
    print('=====================  Vecorizing completed...')
    nbrs = NearestNeighbors(n_neighbors=NEIGHBORS, n_jobs=-1).fit(tfidf)
    query_tf_idf = vectorizer.transform(unique_org)
    distances, indices = nbrs.kneighbors(query_tf_idf)
    return distances, indices

In [24]:
def find_matches(matches,indices, distances, org_name_clean, unique_org, invert= True):
    
    t1 = time.time()
    print('=====================  finding matches...')
    for i,j in enumerate(indices):
        for k in range(0,len(j)):
            str1 = org_name_clean[j[k]].lower()
            str2 = unique_org[i].lower()
            similarity = fuzz.ratio(str1,str2)
            similarity_token_sort_ratio = fuzz.token_sort_ratio(str1,str2)
            if similarity > FILTER_SIMILARITY or similarity_token_sort_ratio > FILTER_SIMILARITY:
                if invert == True:
                    arcus_name = org_name_clean[j[k]]
                    quicken_name = unique_org[i]
                else:                    
                    arcus_name = unique_org[i]
                    quicken_name = org_name_clean[j[k]]
                temp = [round(distances[i][k],2), arcus_name , quicken_name, similarity, similarity_token_sort_ratio]
                matches.append(temp)

    t = time.time() - t1
    print("=====================  COMPLETED IN:", t)
    return matches

## Files Names

In [25]:
file = "data_all.xlsx"
file_quicken = 'full_data_quicken.xlsx'
FILTER_SIMILARITY = 55
NEIGHBORS = 40
ANALIZERS = [
    'char_wb',
    'char',
    'word',
    ngrams
]

## Get the Quicken Dataset

In [26]:
dataset = pd.read_excel(file_quicken, usecols=["Payable To"])
print(dataset.shape, 'dataset completo')
dataset = pd.DataFrame({'name': dataset['Payable To'].unique()})
dataset = dataset
print(dataset.shape, 'dataset con datos unicos')

(345610, 1) dataset completo
(21109, 1) dataset con datos unicos


In [27]:
dataset.head(5)

,name
0,SYNCB/PPC
1,CapitalOne
2,SYNCB/LOW
3,Goldenwest FCU
4,DISCOVER FIN SVCS LLC


## Get Arcus Dataset

In [28]:
payees = pd.read_excel(file,sheet_name='USA-Full List')
payees.dropna(subset=['Name'], inplace=True)
payees['name'] = payees['Name']
payees.drop('Name', axis='columns', inplace=True)
print(payees.shape, " dataset arcus completo")

(16398, 2)  dataset arcus completo


In [29]:
payees.head(5)

,Category,name
0,Airline,American Airlines
1,Airline,United Travel Card
2,Auto Finance,"Ally Financial, Inc."
3,Auto Finance,American Honda Finance Corp
4,Auto Finance,BMW Financial Services


## Copy Quicken Dataset and replace with our manual filter data and lower each string

In [30]:
dataset_client = dataset.copy()
dataset_client.name = dataset_client.apply(lambda row : str_replace(row['name'], str_replaces_list), axis = 1)
dataset_client = dataset_client.name

## Copy Arcus Dataset and replace with our manual filter data and lower each string

In [31]:
dataset_payees = payees.copy()
dataset_payees.name = dataset_payees.apply(lambda row : str_replace(row['name'], str_replaces_list), axis = 1)
dataset_payees = dataset_payees.name

### Match names Arcus vs Quicken with KNeighbors

In [32]:
clean_org_names = dataset_payees
org_name_clean = payees.name
matches = []

In [33]:
print("=====================  Match names Arcus vs Quicken with KNeighbors")
t1 = time.time()
unique_org = dataset_client # set used for increased performance
for i in ANALIZERS:
    distances, indices = train_vectorize(org_name_clean, unique_org, analyzer=i)
    unique_org = dataset.name
    matches = matches + find_matches(matches,indices, distances, org_name_clean, unique_org)
t = time.time() - t1
print("=====================  COMPLETED IN:", t)


=====================  Match names Arcus vs Quicken with KNeighbors
=====================================================Vecorizing the data - this could take a few minutes for large datasets... =====================================================
=====================  COMPLETED IN: 0.3282036781311035
=====================  Vecorizing completed...
=====================  finding matches...
=====================  COMPLETED IN: 25.915809869766235
=====================================================Vecorizing the data - this could take a few minutes for large datasets... =====================================================
=====================  COMPLETED IN: 0.15825390815734863
=====================  Vecorizing completed...
=====================  finding matches...
=====================  COMPLETED IN: 29.937224864959717
=====================================================Vecorizing the data - this could take a few minutes for large datasets... ========================================

In [34]:
print('Building data frame...')  
matches_df = pd.DataFrame(matches, columns=['match confidence(knn)','Arcus name','Quicken name', 'similarity', 'similarity_token_sort_ratio'])
print('Done')

Building data frame...
Done


In [35]:
matches_df.sample(20)

,match confidence(knn),Arcus name,Quicken name,similarity,similarity_token_sort_ratio
4817298,0.59,Sanitary Garbage Company,Grange Property & Casualty Company,52,57
1928347,0.28,Rogue Federal Credit Union,Energy One Federal Credit Union,84,81
152549,0.41,Family Security Credit Union,University of Kentucky Federal Credit Union,54,56
4814016,0.52,Polam FCU,CAP COMM FCU,67,38
3386725,0.52,Lennox Employees Credit Union,PUBLIX EMPLOYEES FED C,63,63
816780,0.37,Coastal Federal Credit Union,HANSCOM FEDERAL CRED,62,62
4538637,0.40,Clermont County Treasurer,Outagamie County Treasurer,75,71
70321,0.46,Charter Township of Canton,CALN TOWNSHIP,56,62
2360117,1.21,First Financial Federal Credit Union,TOPLINE FEDERAL CRD UN,62,59
3927287,0.36,Erie County Water Authority,WRIGHT COUNTY AUDITOR/ TREASURER,58,66


### Match names Quicken vs Arcus with KNeighbors

In [36]:
clean_org_names = dataset_client
org_name_clean = dataset.name

In [37]:
print("=====================  Match names Arcus vs Quicken with KNeighbors")
t1 = time.time()
unique_org = dataset_payees
for i in ANALIZERS:
    distances, indices = train_vectorize(org_name_clean, unique_org, analyzer=i)
    unique_org = payees.name
    matches = matches + find_matches(matches,indices, distances, org_name_clean, unique_org, invert= False)
print("=====================  COMPLETED IN:", t)

=====================  Match names Arcus vs Quicken with KNeighbors
=====================================================Vecorizing the data - this could take a few minutes for large datasets... =====================================================
=====================  COMPLETED IN: 0.35786008834838867
=====================  Vecorizing completed...
=====================  finding matches...
=====================  COMPLETED IN: 18.53573513031006
=====================================================Vecorizing the data - this could take a few minutes for large datasets... =====================================================
=====================  COMPLETED IN: 0.3540341854095459
=====================  Vecorizing completed...
=====================  finding matches...
=====================  COMPLETED IN: 18.916866779327393
=====================================================Vecorizing the data - this could take a few minutes for large datasets... =========================================

In [38]:
print('Building data frame...')  
matches_df = pd.DataFrame(matches, columns=['match confidence(knn)','Arcus name','Quicken name', 'similarity', 'similarity_token_sort_ratio'])
print('Done')

Building data frame...
Done


In [39]:
matches_df.sample(20, random_state=42)

,match confidence(knn),Arcus name,Quicken name,similarity,similarity_token_sort_ratio
31310116,0.49,City of Cranston,TRENTON CITY,43,71
81607092,0.65,Launch Servicing LLC,LVNV FUNDING LLC,56,50
11688692,0.41,Crane FCU,SCE FCU,75,38
14964815,0.35,Humana Insurance Company,American Family Mutual Insurance Company,69,59
8539621,0.43,Sunset Life Insurance Company,SAFETY INDEMNITY INSURANCE,58,62
4702346,0.52,Infirst Federal Credit Union Loan,FORT SILL FED CR UN,50,58
58232441,0.58,Treana Community Association,CASSIA COUNTY,44,59
66095947,0.69,DESCO FCU,SECPLUS FCU,60,40
5367265,0.44,Kansas State Bank Loan,NORTHEAST BANK,50,56
11644659,0.34,Franklin County Treasurer,LAKE COUNTY TREASURER,83,83


## Create the dataset result and test some Quicken names

In [40]:
matches_df = matches_df.drop_duplicates(subset=['Arcus name', 'Quicken name'])
matches_df.shape

(417500, 5)

In [41]:
matches_df[matches_df['similarity_token_sort_ratio'] > 65]

,match confidence(knn),Arcus name,Quicken name,similarity,similarity_token_sort_ratio
0,0.31,Capital One,CapitalOne,95,95
2,0.45,Old West FCU,Goldenwest FCU,85,85
18,0.36,Brauns Capital One RS,CAPITAL ONE BANK USA N,60,79
43,0.00,Barclays Bank Delaware,BARCLAYS BANK DELAWARE,100,100
47,0.49,Barclays Bank Consumer Loans,BARCLAYS BANK DELAWARE,68,68
...,...,...,...,...,...
41740483,0.97,Water And Power Community Credit Union,OUR COMMUNITY CREDIT,66,69
41740485,0.99,Water And Power Community Credit Union,U of VA Community Credit Union,74,76
41740502,0.97,Westerly Community Credit Union Loan,HAPO COMMUNITY CREDIT UNION,75,76
41740509,1.00,Winslow Community FCU Loan,UTAH COMMUNITY FCU,64,68


In [42]:
capital_one =  matches_df[matches_df['Quicken name'] == 'CAPITAL ONE BANK USA N']
capital_one[capital_one['similarity_token_sort_ratio'] > 90]
capital_one.sort_values('similarity_token_sort_ratio', ascending=False)

,match confidence(knn),Arcus name,Quicken name,similarity,similarity_token_sort_ratio
18,0.36,Brauns Capital One RS,CAPITAL ONE BANK USA N,60,79
5121382,0.41,Amana Capital One RS,CAPITAL ONE BANK USA N,62,76
5121667,0.44,BNB RS CAPITAL ONE RS,CAPITAL ONE BANK USA N,60,74
1072771,1.06,BJ's Capital One RS,CAPITAL ONE BANK USA N,63,73
1072759,0.97,"Capital One, NA",CAPITAL ONE BANK USA N,76,72
...,...,...,...,...,...
359135,0.50,Pinnacle Bank Nebraska,CAPITAL ONE BANK USA N,59,50
17,0.35,American State Bank Loan,CAPITAL ONE BANK USA N,57,48
5063917,0.46,ACNB Bank Loan,CAPITAL ONE BANK USA N,56,44
5126334,0.43,Associated Bank Loan,CAPITAL ONE BANK USA N,57,43


In [68]:
capital_one = matches_df[matches_df['Quicken name'] == 'CAPITAL ONE']
capital_one[capital_one['similarity_token_sort_ratio'] > 85]
capital_one.sort_values('similarity_token_sort_ratio', ascending=False).drop_duplicates(subset=['Arcus name', 'Quicken name']).head(1)

,match confidence(knn),Arcus name,Quicken name,similarity,similarity_token_sort_ratio
345,0.0,Capital One,CAPITAL ONE,100,100


In [69]:
matches_df[matches_df['Quicken name'] == 'BARCLAYS BANK DELAWA'].sort_values('similarity_token_sort_ratio', ascending=False).head(1)

,match confidence(knn),Arcus name,Quicken name,similarity,similarity_token_sort_ratio
1478,0.14,Barclays Bank Delaware,BARCLAYS BANK DELAWA,95,95


In [70]:
matches_df[matches_df['Quicken name'] == 'AMERICAN HONDA FINANCE'].sort_values('similarity_token_sort_ratio', ascending=False).head(1)

,match confidence(knn),Arcus name,Quicken name,similarity,similarity_token_sort_ratio
1545,0.25,American Honda Finance Corp,AMERICAN HONDA FINANCE,90,90


In [71]:
matches_df[matches_df['Quicken name'] == 'BANK OF AMER'].sort_values('similarity_token_sort_ratio', ascending=False).head(1)

,match confidence(knn),Arcus name,Quicken name,similarity,similarity_token_sort_ratio
16925,0.38,Bank of America - AFS,BANK OF AMER,73,77


In [72]:
matches_df[matches_df['Quicken name'] == 'Bank of America'].sort_values('similarity_token_sort_ratio', ascending=False).head(1)

,match confidence(knn),Arcus name,Quicken name,similarity,similarity_token_sort_ratio
56596,0.35,Bank of America - AFS,Bank of America,83,88


In [75]:
bank = matches_df[matches_df['Quicken name'] == 'BANK OF AMER']
bank.sort_values(by=['similarity_token_sort_ratio'], ascending=False).head(1)

,match confidence(knn),Arcus name,Quicken name,similarity,similarity_token_sort_ratio
16925,0.38,Bank of America - AFS,BANK OF AMER,73,77


In [49]:
amica_mutual = matches_df[matches_df['Arcus name'] == 'Amica Mutual Insurance Company']
amica_mutual.sort_values(by='similarity_token_sort_ratio', ascending=False)

,match confidence(knn),Arcus name,Quicken name,similarity,similarity_token_sort_ratio
143783,0.00,Amica Mutual Insurance Company,Amica Mutual INsurance Company,100,100
175302,0.00,Amica Mutual Insurance Company,AMICA MUTUAL INSURANCE COMPANY,100,100
38721,0.00,Amica Mutual Insurance Company,Amica Mutual Insurance Company,100,100
44697,0.25,Amica Mutual Insurance Company,Alfa Mutual Insurance Company,92,92
161722,0.25,Amica Mutual Insurance Company,ALFA MUTUAL INSURANCE COMPANY,92,92
...,...,...,...,...,...
119405,0.42,Amica Mutual Insurance Company,MUNICIPAL MUTUAL INS,60,52
138638,0.39,Amica Mutual Insurance Company,MUSCATINE MUTUAL INS ASS OCIATION,60,51
51703,0.48,Amica Mutual Insurance Company,WAWANESA MUTUAL INS CO,62,50
125017,0.46,Amica Mutual Insurance Company,Farm Bureau Casualty Insurance,57,50


In [50]:
matches_df[matches_df['Quicken name'] == 'Partners Federal CU']

,match confidence(knn),Arcus name,Quicken name,similarity,similarity_token_sort_ratio
106944,0.31,Partners Federal Credit Union,Partners Federal CU,79,75
106945,0.38,Apple Federal Credit Union,Partners Federal CU,58,49
106946,0.38,Star USA Federal Credit Union,Partners Federal CU,58,58
106947,0.38,Realtors Federal Credit Union,Partners Federal CU,62,58
106948,0.38,APL Federal Credit Union,Partners Federal CU,56,51
106949,0.40,Strata Federal Credit Union,Partners Federal CU,57,57
106950,0.40,Altra Federal Credit Union,Partners Federal CU,62,49
106951,0.40,Fortera Federal Credit Union,Partners Federal CU,64,60
106952,0.41,CP Federal Credit Union,Partners Federal CU,57,52
106953,0.41,Harvesters Federal Credit Union Loan,Partners Federal CU,62,58


In [51]:
len(matches_df['Quicken name'].unique())

19341

In [52]:
len(matches_df['Quicken name'])

417500

In [53]:
len(matches_df['Arcus name'].unique())

14933

In [54]:
len(matches_df['Arcus name'])

417500

In [55]:
result = payees.merge(matches_df, left_on='name', right_on='Arcus name', how='inner')

In [56]:
amica_mutual = result[result['Arcus name'] == 'Amica Mutual Insurance Company']
amica_mutual.sort_values(by='similarity_token_sort_ratio', ascending=False)

,Category,name,match confidence(knn),Arcus name,Quicken name,similarity,similarity_token_sort_ratio
216840,Insurance,Amica Mutual Insurance Company,0.00,Amica Mutual Insurance Company,Amica Mutual INsurance Company,100,100
216896,Insurance,Amica Mutual Insurance Company,0.00,Amica Mutual Insurance Company,AMICA MUTUAL INSURANCE COMPANY,100,100
216678,Insurance,Amica Mutual Insurance Company,0.00,Amica Mutual Insurance Company,Amica Mutual Insurance Company,100,100
216686,Insurance,Amica Mutual Insurance Company,0.25,Amica Mutual Insurance Company,Alfa Mutual Insurance Company,92,92
216872,Insurance,Amica Mutual Insurance Company,0.25,Amica Mutual Insurance Company,ALFA MUTUAL INSURANCE COMPANY,92,92
...,...,...,...,...,...,...,...
216806,Insurance,Amica Mutual Insurance Company,0.42,Amica Mutual Insurance Company,MUNICIPAL MUTUAL INS,60,52
216833,Insurance,Amica Mutual Insurance Company,0.39,Amica Mutual Insurance Company,MUSCATINE MUTUAL INS ASS OCIATION,60,51
216699,Insurance,Amica Mutual Insurance Company,0.48,Amica Mutual Insurance Company,WAWANESA MUTUAL INS CO,62,50
216814,Insurance,Amica Mutual Insurance Company,0.46,Amica Mutual Insurance Company,Farm Bureau Casualty Insurance,57,50


In [57]:
print(len(matches_df['Quicken name'].unique()), ' quicken unique')
print(len(matches_df['Arcus name'].unique()), ' arcus unique')
print(matches_df.shape, ' all data')

19341  quicken unique
14933  arcus unique
(417500, 5)  all data


In [58]:
print(len(matches_df['Quicken name'].unique()), ' quicken unique')
print(len(matches_df['Arcus name'].unique()), ' arcus unique')
print(matches_df.shape, ' all data')

19341  quicken unique
14933  arcus unique
(417500, 5)  all data


In [59]:
sim_m_75 = matches_df[matches_df['similarity'] > 60]

In [60]:
len(sim_m_75['Quicken name'].unique())

16139

In [61]:
sim_a_75 = matches_df[(matches_df['match confidence(knn)'] < 1) & (matches_df['match confidence(knn)'] > 0 )]

In [62]:
sim_a_75.sort_values(by=['match confidence(knn)', 'similarity'], ascending=True)

,match confidence(knn),Arcus name,Quicken name,similarity,similarity_token_sort_ratio
70474,0.07,New Jersey Manufacturers Insurance Company,NEW JERSEY MANUFACTURES INSURANCE COMPANY,99,99
143165,0.07,Michigan Basic Property Insurance Assoc,Michigan Basic Property Insurance ASSc,99,99
137424,0.08,Indiana Farm Bureau Insurance,Indian Farm Bureau Insurance,98,98
154770,0.08,Xcel Federal Credit Union,EXCEL Federal Credit Union,98,78
175188,0.08,Universal Property & Casualty,Universal Property & Casualy,98,98
...,...,...,...,...,...
1173545,0.99,Harris County MUD 188,HARRIS COUNTY MUD 153,90,90
1173546,0.99,Harris County MUD 127,HARRIS COUNTY MUD 153,90,90
1173547,0.99,Harris County MUD 261,HARRIS COUNTY MUD 153,90,90
1173549,0.99,Harris County MUD 162,HARRIS COUNTY MUD 153,90,90


In [63]:
matches_df.to_excel("matched_names_tf_idf.xlsx", engine="xlsxwriter", index=False)
matches_df.to_csv("matched_names_tf_idf.csv", index=False)

In [64]:
matches_df.shape

(417500, 5)

In [65]:
print(len(matches_df['Quicken name'].unique()), ' quicken unique')
print(len(matches_df['Arcus name'].unique()), ' arcus unique')
print(matches_df.shape, ' all data')

19341  quicken unique
14933  arcus unique
(417500, 5)  all data


In [66]:
nNames = len(matches_df['Quicken name'].unique())
print("Total dataset", dataset_client.shape[0])
print("Total Names found ", nNames)
print("Percent : ", 100 * nNames / dataset_client.shape[0], '%')

Total dataset 21109
Total Names found  19341
Percent :  91.62442560045479 %


In [248]:
m_m_80 = matches_df[matches_df['similarity_token_sort_ratio'] > 80]
m_m_80.shape

(13104, 5)

In [249]:
m_m_85 = matches_df[matches_df['similarity_token_sort_ratio'] > 85]

In [250]:
m_m_95 = matches_df[matches_df['similarity_token_sort_ratio'] > 90]

In [251]:
len(m_m_80['Quicken name'].unique())

4705

In [295]:
df = m_m_80[m_m_80.groupby('Quicken name')['similarity_token_sort_ratio'].transform('max') == m_m_80['similarity_token_sort_ratio']]
print(len(df))

5119


In [282]:
m_m_80.groupby('Quicken name')['similarity_token_sort_ratio'].transform('max')


0            95
2            85
43          100
53           83
84           88
           ... 
41738665    100
41738694    100
41738697    100
41738702     92
41738709    100
Name: similarity_token_sort_ratio, Length: 13104, dtype: int64

In [286]:
m_m_80.loc[41738665]

match confidence(knn)                                 1
Arcus name                     GFA Federal Credit Union
Quicken name                    CP FEDERAL CREDIT UNION
similarity                                           89
similarity_token_sort_ratio                          85
Name: 41738665, dtype: object